<img src=images/xd-logo.png align=right width=300px>

# Testing FastAPI

> When it comes to writing APIs, possibly the most important **best-practice** to follow is to **separate the code that implements the logic of your program from the code that exposes your program as an API.**

In this example the logic of our program intertwined with the endpoint definitions. This forces you to have to interact with the API if you want to test the logic of your program, which is not ideal.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

from typing import Dict

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: str | None = None

items: Dict[str, Item] = {}

@app.post("/items/")
async def create_item(item: Item):
    item_dict = item.dict()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    items[item.name] = item_dict
    return item_dict

@app.get("/item/{item_name}")
async def serve_item(item_name: str):
    if item_name in items:
        item = items[item_name]
        return item
    else:
        raise HTTPException(404, f"can't find {item_name}")

Instead, let's separate the logic of the program into functions that are independent from the endpoint definitions.

Even better would be to separate the data models and program logic into a python package, and create the API as an additional package that imports the program logic and data structures. But let's keep it simple for now and create a new file called `app.py` with all the code from the cell below.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

from typing import Dict

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None

items: Dict[str, Item] = {}

def update_items(item: Item) -> dict:
    item_dict = item.model_dump()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    items[item.name] = item_dict
    return item_dict

def get_item(item_name: str) -> dict | None:
    if item_name in items:
        return items[item_name]
    return None

app = FastAPI()

@app.post("/items/")
async def create_item(item: Item):
    return update_items(item)

@app.get("/item/{item_name}")
async def serve_item(item_name: str):
    item = get_item(item_name)
    if not item:
        raise HTTPException(404, f"can't find {item_name}")
    return item

And now it's possible to test separatedly the behaviour of our program and the behaviour of our API endpoints.

To test the endpoints we have two approaches:
 - Using FastAPI's `TestClient` that allows you to test the API without having to start a webserver.
 - Or by mocking the API connection.

You can create a new file called `tests.py` with the following code to try it out.

In [ ]:
from fastapi.testclient import TestClient
from unittest import mock

# Uncomment this
# from app import app, items, Item, update_items

# Test the program logic
def test_update_items():
    item = Item(
        name="candy",
        description="nice chocolates",
        price=5,
        tax=1
    )

    res = update_items(item)
    assert res['price_with_tax'] == 6

# Test the endpoints
client = TestClient(app)

def test_get_item_endoint():
   items['foo'] = {"my":"fake item"}

   res = client.get("/item/foo")
   assert res.status_code == 200
   assert res.json() == {"my":"fake item"}

# Test a mocked-up endpoint
def test_get_item_mocked():
    with mock.patch('app.get_item', return_value={'my': 'faked fake item'}):
        res = client.get("/item/foo")
        assert res.status_code == 200
        assert res.json() == {"my": "faked fake item"}

Now you can run the tests with `pytest tests.py`.

Some exercises:
- Can you try adding some more tests?
- Edit `app.py` to use Pydantic to filter the output of `API/item/{{ITEM}}` so that it doesn't return the `price` and `tax`, but only the `price_with_tax`.
- Test this functionality.